# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.123428e+02     1.653033e+00
 * time: 0.6367080211639404
     1     1.096594e+01     9.343237e-01
 * time: 1.7664189338684082
     2    -1.166296e+01     1.007947e+00
 * time: 2.313917875289917
     3    -3.387546e+01     8.173258e-01
 * time: 3.1346569061279297
     4    -4.743878e+01     6.157458e-01
 * time: 3.951097011566162
     5    -5.680087e+01     3.016789e-01
 * time: 4.767641067504883
     6    -5.972159e+01     1.786478e-01
 * time: 5.303225994110107
     7    -6.085105e+01     5.281289e-02
 * time: 5.848006963729858
     8    -6.126022e+01     9.337059e-02
 * time: 6.386405944824219
     9    -6.153086e+01     4.265752e-02
 * time: 6.931034088134766
    10    -6.174703e+01     3.211662e-02
 * time: 7.4799299240112305
    11    -6.193125e+01     2.800318e-02
 * time: 8.017701864242554
    12    -6.200565e+01     1.888394e-02
 * time: 8.55596399307251
    13    -6.208092e+01     1.762483e-02
 * time: 9.10153603553772
    1

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671066
    AtomicLocal         -18.8557675
    AtomicNonlocal      14.8522647
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485369 
    Xc                  -19.3336819

    total               -62.261666462064
